<a href="https://colab.research.google.com/github/Obolrom/DSL/blob/main/Job2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from copy import deepcopy
"""
Task:
    Remove all useless non-terminal symbols, 
    and identify nullable (vanishing) non-terminal symbols.

{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token: (class, value)
class: int
value: str
var: str                 # name of the non-terminal symbol
definition: list(rule)
rule: list(var | token)  # right part of the rule
"""

"""
To start with, we will use the following context free grammar structure:

Steps to success:
    1. get all productive (generating) symbols
    2. get all reachable symbols
    3. remove all useless non-terminal symbols
    4. get all nullable (vanishing) symbols
"""


VARS = "vars"
TOKENS = "toks"
H_VAR = "hvar"


def productive_symbols(grammar) -> set:
    productive = set()
    prev_count = None

    def absent_unproductive_symbols(rule: str) -> bool:
        return all(map(lambda s: s in productive or s in grammar[TOKENS], rule))

    while len(productive) != prev_count:
        prev_count = len(productive)
        productive_symbols = [var for var, definition in grammar[VARS].items() if
                              list(filter(absent_unproductive_symbols, definition)) != []]
        productive = productive.union(set(productive_symbols))

    return productive


def reachable_symbols(grammar) -> set:
    reachable = {grammar[H_VAR]}
    prev_count = None

    while len(reachable) != prev_count:
        prev_count = len(reachable)
        for var, definition in grammar[VARS].items():
            if var in reachable:
                for rule in definition:
                    reachable = reachable.union(set(filter(lambda s: s not in grammar[TOKENS], rule)))

    return reachable


def remove_useless_symbols(grammar):
    """
    Removes all rules with unproductive or unreachable symbols.
    """
    grammar_copy = deepcopy(grammar)

    productive = productive_symbols(grammar)
    grammar_copy[VARS] = {
        var: [rule for rule in definition if all(map(lambda s: s in productive or s in grammar[TOKENS], rule))]
        for var, definition in grammar_copy[VARS].items()}

    reachable = reachable_symbols(grammar_copy)
    grammar_copy[VARS] = {
        var: [rule for rule in definition if all(map(lambda s: s in reachable or s in grammar[TOKENS], rule))]
        for var, definition in grammar_copy[VARS].items() if var in reachable}

    return grammar_copy


def nullable_symbols(grammar) -> set:
    nullable = set()
    prev_count = None

    def all_nullable_symbols(rule: str) -> bool:
        return all(map(lambda s: s in nullable or s == (0, ''), rule))

    while len(nullable) != prev_count:
        prev_count = len(nullable)
        nullable_symbols = [var for var, definition in grammar[VARS].items()
                            if list(filter(all_nullable_symbols, definition)) != []]
        nullable = nullable.union(set(nullable_symbols))

    return nullable


if __name__ == '__main__':
    grammar = {
        TOKENS: {
            (0, ''),
            (1, 'a'),
            (2, 'b'),
            (3, 'c')
        },
        VARS: {
            'S': [[(1, 'a'), 'M'],
                  [(1, 'a'), 'S'],
                  ['K']],
            'K': [[(3, 'c')],
                  [(0, '')]],
            'M': [['M', 'P']],
            'P': [[(3, 'c')]]
        },
        H_VAR: 'S'
    }

    grammar = remove_useless_symbols(grammar)
    print(grammar, end='\n\n')
    print(f"Nullable symbols: {nullable_symbols(grammar)}")


{'toks': {(1, 'a'), (0, ''), (3, 'c'), (2, 'b')}, 'vars': {'S': [[(1, 'a'), 'S'], ['K']], 'K': [[(3, 'c')], [(0, '')]]}, 'hvar': 'S'}

Nullable symbols: {'K', 'S'}
